In [1]:
# Load fine-tuned model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gokuls/BERT-tiny-emotion-intent")
model = AutoModelForSequenceClassification.from_pretrained("gokuls/BERT-tiny-emotion-intent")

/home/tony/FHE-BERT-Tiny/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tony/FHE-BERT-Tiny/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import os
import numpy as np

# Create a directory to save the parameters
os.makedirs('weights-emotion', exist_ok=True)

# Access the state dictionary of the model
state_dict = model.state_dict()

# Function to save a tensor to a text file
def save_tensor_to_txt(tensor, filepath):
    # Convert tensor to numpy array
    array = tensor.cpu().numpy()
    # Save to text file with desired formatting
    with open(filepath, 'w') as f:
        for row in array.reshape(-1):
            f.write(f"{row}\n")
            
# Function to save a tensor to a text file
def save_tensor_to_txt2(tensor, filepath):
    # Convert tensor to numpy array
    array = tensor.cpu().numpy()
    # Reshape the array to 128 rows and 128 columns
    reshaped_array = array.reshape(128, 128)
    # Save to text file with desired formatting
    np.savetxt(filepath, reshaped_array, delimiter=',')

exclude_keywords = [
    "attself_query_weight",
    "attself_key_weight",
    "attself_value_weight",
    "selfouotput_weight"
    "pooler_dense_weight"
]
    
# Iterate through all parameters and save the ones you're interested in
for name, param in state_dict.items():
    # Check if the parameter name matches the desired layers
    if any(keyword in name for keyword in [
        'attention.self.query', 
        'attention.self.key', 
        'attention.self.value',
        'attention.output.dense',
        'intermediate.dense',
        'output.dense',
        'pooler.dense',
        'classifier'
    ]):
        # Replace dots with underscores for file naming
        filename = name.replace('.', '_') + '.txt'
        
        # remove bert_encoder_
        if filename[:13] == "bert_encoder_":
            filename = filename[13:]
        elif filename[:5] == "bert_":
            filename = filename[5:]
            
        # layer_0 -> layer0
        filename = filename.replace('layer_0', 'layer0')
        filename = filename.replace('layer_1', 'layer1')
        
        # attention_self -> attself
        # attention_output_dense -> selfoutput
        # intermediate_dense -> intermediate
        # output_dense -> output
        filename = filename.replace('attention_self', 'attself')
        filename = filename.replace('attention_output_dense', 'selfoutput')
        filename = filename.replace('intermediate_dense', 'intermediate')
        filename = filename.replace('output_dense', 'output')
        
        filepath = os.path.join('weights-emotion', filename)
        
        if any(keyword in filepath for keyword in exclude_keywords):
            save_tensor_to_txt2(param, filepath)
        else:
            save_tensor_to_txt(param, filepath)
        print(f"Saved {name} to {filepath}")

Saved bert.encoder.layer.0.attention.self.query.weight to weights-emotion/layer0_attself_query_weight.txt
Saved bert.encoder.layer.0.attention.self.query.bias to weights-emotion/layer0_attself_query_bias.txt
Saved bert.encoder.layer.0.attention.self.key.weight to weights-emotion/layer0_attself_key_weight.txt
Saved bert.encoder.layer.0.attention.self.key.bias to weights-emotion/layer0_attself_key_bias.txt
Saved bert.encoder.layer.0.attention.self.value.weight to weights-emotion/layer0_attself_value_weight.txt
Saved bert.encoder.layer.0.attention.self.value.bias to weights-emotion/layer0_attself_value_bias.txt
Saved bert.encoder.layer.0.attention.output.dense.weight to weights-emotion/layer0_selfoutput_weight.txt
Saved bert.encoder.layer.0.attention.output.dense.bias to weights-emotion/layer0_selfoutput_bias.txt
Saved bert.encoder.layer.0.intermediate.dense.weight to weights-emotion/layer0_intermediate_weight.txt
Saved bert.encoder.layer.0.intermediate.dense.bias to weights-emotion/layer

In [3]:
import numpy as np

layer0_intermediate_weight = np.loadtxt(f"./weights-emotion/layer0_intermediate_weight.txt")
layer1_intermediate_weight = np.loadtxt(f"./weights-emotion/layer1_intermediate_weight.txt")

layer0_output_weight = np.loadtxt(f"./weights-emotion/layer0_output_weight.txt")
layer1_output_weight = np.loadtxt(f"./weights-emotion/layer1_output_weight.txt")

In [4]:
split_size = len(layer0_intermediate_weight)/4
num = int(split_size)

layer0_intermediate_weight1 = layer0_intermediate_weight[:num]
layer0_intermediate_weight2 = layer0_intermediate_weight[num:(num*2)]
layer0_intermediate_weight3 = layer0_intermediate_weight[(num*2):(num*3)]
layer0_intermediate_weight4 = layer0_intermediate_weight[(num*3):(num*4)]
layer1_intermediate_weight1 = layer1_intermediate_weight[:num]
layer1_intermediate_weight2 = layer1_intermediate_weight[num:(num*2)]
layer1_intermediate_weight3 = layer1_intermediate_weight[(num*2):(num*3)]
layer1_intermediate_weight4 = layer1_intermediate_weight[(num*3):(num*4)]

layer0_output_weight1 = layer0_output_weight[:num]
layer0_output_weight2 = layer0_output_weight[num:(num*2)]
layer0_output_weight3 = layer0_output_weight[(num*2):(num*3)]
layer0_output_weight4 = layer0_output_weight[(num*3):(num*4)]
layer1_output_weight1 = layer1_output_weight[:num]
layer1_output_weight2 = layer1_output_weight[num:(num*2)]
layer1_output_weight3 = layer1_output_weight[(num*2):(num*3)]
layer1_output_weight4 = layer1_output_weight[(num*3):(num*4)]

np.savetxt("./weights-emotion/layer0_intermediate_weight1.txt", layer0_intermediate_weight1)
np.savetxt("./weights-emotion/layer0_intermediate_weight2.txt", layer0_intermediate_weight2)
np.savetxt("./weights-emotion/layer0_intermediate_weight3.txt", layer0_intermediate_weight3)
np.savetxt("./weights-emotion/layer0_intermediate_weight4.txt", layer0_intermediate_weight4)
np.savetxt("./weights-emotion/layer1_intermediate_weight1.txt", layer1_intermediate_weight1)
np.savetxt("./weights-emotion/layer1_intermediate_weight2.txt", layer1_intermediate_weight2)
np.savetxt("./weights-emotion/layer1_intermediate_weight3.txt", layer1_intermediate_weight3)
np.savetxt("./weights-emotion/layer1_intermediate_weight4.txt", layer1_intermediate_weight4)

In [5]:
os.remove("./weights-emotion/layer0_intermediate_weight.txt")
os.remove("./weights-emotion/layer1_intermediate_weight.txt")

os.remove("./weights-emotion/layer0_output_weight.txt")
os.remove("./weights-emotion/layer1_output_weight.txt")

In [6]:
print(len(layer0_intermediate_weight1))
print(len(layer0_intermediate_weight2))
print(len(layer0_intermediate_weight3))
print(len(layer0_intermediate_weight4))

16384
16384
16384
16384


In [7]:
print(layer0_intermediate_weight1[16383])
print(layer0_intermediate_weight2[0])
print(layer0_intermediate_weight2[16383])
print(layer0_intermediate_weight3[16383])
print(layer0_intermediate_weight4[16383])

-0.1657962054014206
0.000812145066447556
-0.007029421627521515
-0.08165649324655533
0.1562705934047699


In [8]:
print(layer0_intermediate_weight[16383])
print(layer0_intermediate_weight[16384])
print(layer0_intermediate_weight[32767])
print(layer0_intermediate_weight[49151])
print(layer0_intermediate_weight[65535])

-0.1657962054014206
0.000812145066447556
-0.007029421627521515
-0.08165649324655533
0.1562705934047699
